## Set runtime python2 with GPU

In [0]:
import os
import shutil

## Clone the repository

In [0]:
!git clone --recursive https://github.com/mahyarnajibi/SSH.git

## Installing dependencies

In [0]:
!apt-get install libprotobuf-dev libleveldb-dev libsnappy-dev libopencv-dev libhdf5-serial-dev protobuf-compiler

In [0]:
!apt-get install --no-install-recommends libboost-all-dev

In [0]:
!apt-get install libgflags-dev libgoogle-glog-dev liblmdb-dev libhdf5-serial-dev

In [0]:
!apt-get install libhdf5-dev libhdf5-serial-dev

In [0]:
!apt install libnccl2 libnccl-dev

In [0]:
!apt-get install libopencv-dev

## Make hdf5 links for caffe sees them



In [0]:
os.chdir('/usr/lib/x86_64-linux-gnu')

In [0]:
!ln -s libhdf5_serial.so.100.0.1 libhdf5.so

In [0]:
!ln -s libhdf5_serial_hl.so.100.0.0 libhdf5_hl.so

## Fix Makefile

In [0]:
os.chdir('/content/SSH/caffe-ssh')

In [0]:
os.rename('./Makefile.config.example', './Makefile.config')

In [0]:
with open('./Makefile.config','r') as makefile:
  lines = makefile.readlines()
target_string_index = lines.index('INCLUDE_DIRS := $(PYTHON_INCLUDE) /usr/local/include\n')
lines[target_string_index] = 'INCLUDE_DIRS := $(PYTHON_INCLUDE) /usr/local/include /usr/include/hdf5/serial/\n'
with open('./Makefile.config','w') as makefile:
  makefile.write("".join(lines))

In [0]:
with open('./Makefile.config','r') as makefile:
  lines = makefile.readlines()
target_string_index = lines.index('USE_NCCL := 1\n')
lines[target_string_index] = 'USE_NCCL := 0\n'
with open('./Makefile.config','w') as makefile:
  makefile.write("".join(lines))

In [0]:
with open('./Makefile.config','r') as makefile:
  lines = makefile.readlines()
lines.remove('CUDA_ARCH := -gencode arch=compute_20,code=sm_20 \\\n')
lines.remove('\t\t-gencode arch=compute_20,code=sm_21 \\\n')
target_string_index = lines.index('\t\t-gencode arch=compute_30,code=sm_30 \\\n')
lines[target_string_index] = 'CUDA_ARCH := -gencode arch=compute_30,code=sm_30 \\\n -gencode arch=compute_37,code=sm_37 \\\n'
with open('./Makefile.config','w') as makefile:
  makefile.write("".join(lines))

In [0]:
with open('./Makefile.config','r') as makefile:
  lines = makefile.readlines()
target_string_index = lines.index('# OPENCV_VERSION := 3\n')
lines[target_string_index] = 'OPENCV_VERSION := 3\n'
with open('./Makefile.config','w') as makefile:
  makefile.write("".join(lines))

In [0]:
with open('./Makefile.config','r') as makefile:
  lines = makefile.readlines()
target_string_index = lines.index('\t\t/usr/lib/python2.7/dist-packages/numpy/core/include\n')
lines[target_string_index] = '\t\t /usr/local/lib/python2.7/dist-packages/numpy/core/include\n'
with open('./Makefile.config','w') as makefile:
  makefile.write("".join(lines))

## Make all

In [0]:
!make all

## Make tests

In [0]:
!make test

## Run tests

In [0]:
!make runtest

## Make distribute

In [0]:
!make distribute

## Move distribute

In [0]:
!cp -R /content/SSH/caffe-ssh/distribute/* /usr

## Add path

In [0]:
import sys
sys.path.append('/usr/python')

## Make model

In [0]:
os.chdir('/content/SSH')

In [0]:
!pip install -r requirements.txt

In [0]:
os.chdir('/content/SSH/lib')

In [0]:
!make

## Download models

In [0]:
os.chdir('/content/SSH')

In [0]:
!bash scripts/download_ssh_model.sh

## Run

In [0]:
!python demo.py